## Imports

In [ ]:
%load_ext lab_black

import h5py
import os
import numpy as np
from typing import Dict, List, Optional, Tuple

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache


import plotly.graph_objects as go
import plotly.colors as pc
import matplotlib.pyplot as plt


import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import (
    fitting_functions,
    analysis_functions,
    plotting_functions,
    HDF5_loader,
    misc_functions,
    filter_functions,
    tr_functions,
    loading_functions,
    kw_data_loader,
    cnn,
    polygons,
)

colors = pc.qualitative.D3
angstrom = "\u212B"
Theta = "\u0398"
phi = "\u03C6"

# Convert Data

In [ ]:
## Convert K corrected .ibw data to .h5 ##

ddir = r"E:\atully\arpes_data\2022_April\ARPES\C60\averaged_data"

# STEP 1 ##
# Convert ibw to hdf5
fn = "FS5_avg.ibw"
HDF5_loader.ibw_to_hdf5(ddir, fn, export=True)

# Check conversion worked
data, kx, ky, energy = HDF5_loader.load_hdf5(ddir, "FS5_avg.h5")  # load data from hdf5
data.shape, kx.shape, ky.shape, energy.shape

# Load Data

In [ ]:
files = []
files = ["FS5_avg.h5"]

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=kx, phi_or_time=ky, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad = ARPES_DATA[files[0]]

In [ ]:
slice_val = 17.7

slice_dim = "y"
int_range = 0.1
xlim = (-21, 17)
# xlim = None
ylim = None
x_bin = 1
y_bin = 1

In [ ]:
title = f"Damaged Film: (E<sub>B</sub> = {np.round(slice_val - 18.4, 1)} eV)"
yaxis_title = f"{phi}"
xaxis_title = f"{Theta}"

In [ ]:
## Get Slices ##

x, y, d = tr_functions.slice_datacube(
    ad_dataclass=ad,
    slice_dim=slice_dim,
    slice_val=slice_val,
    int_range=int_range,
    xlim=xlim,
    ylim=ylim,
    x_bin=x_bin,
    y_bin=y_bin,
    norm_data=True,
    plot_data=False,
)

In [ ]:
## Plot Data ##

x_plot, y_plot, d_plot = x, y, d

fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    height=500,
    width=500,
)

fig.add_trace(
    go.Heatmap(
        x=x_plot,
        y=y_plot,
        z=analysis_functions.norm_data(d_plot),
        coloraxis="coloraxis",
    )
)

fig.update_coloraxes(cmin=0, cmax=1)
# fig.update_coloraxes(colorscale="Blues", reversescale=False)

fig.show()